In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
from urllib.request import urlopen, Request

In [2]:
r = Request('https://th.investing.com/economic-calendar/', headers={'User-Agent': 'Mozilla/5.0'})
time.sleep(2)
response = urlopen(r).read()
soup = BeautifulSoup(response, "html.parser")
table = soup.find_all(class_ = "js-event-item")
result = []

In [3]:
def event_calendar():
    
    for bl in table:
        event_datetime = bl.get('data-event-datetime', '').strip()
        currency = bl.find(class_="left flagCur noWrap").text.strip()
        intensity_divs = bl.find_all(class_="left textNum sentiment noWrap")
        event = bl.find(class_="left event").text.strip()
        intencity_val = 0
        true_count = 0

        for intence in intensity_divs:
            _true = intence.find_all(class_="grayFullBullishIcon")
            _false = intence.find_all(class_="grayEmptyBullishIcon")

            true_count = len(_true)

            if true_count == 3:
                intencity_val = 3
            elif true_count == 2:
                intencity_val = 2
            else :
                intencity_val = 1
                
        event_datetime = event_datetime.split(' ')
        date=event_datetime[0]
        time=event_datetime[1]
        
        
        result.append({'currency' : currency, 'date' : date, 'time' : time, 'intensity' : intencity_val, 'event':event})

    return result

In [4]:
news = event_calendar()
news_df = pd.DataFrame(news)

In [5]:
display(news_df.shape)
display(news_df.head(10))

(59, 5)

,currency,date,time,intensity,event
0,USD,2025/05/21,03:30:00,2,รายงานสินค้าคงเหลือของน้ำมันดิบประจำไตรมาสจาก API
1,USD,2025/05/21,04:00:00,1,สุนทรพจน์ของฟิสเชอร์ (Fisher) สมาชิกคณะกรรมการ...
2,NZD,2025/05/21,05:45:00,1,ปริมาณการส่งออกสินค้า (เม.ย.)
3,NZD,2025/05/21,05:45:00,1,ปริมาณการนำเข้าสินค้า (เม.ย.)
4,NZD,2025/05/21,05:45:00,2,ดุลการค้า (เดือนต่อเดือน) (เม.ย.)
5,NZD,2025/05/21,05:45:00,2,ดุลการค้า (ปีต่อปี) (เม.ย.)
6,USD,2025/05/21,06:00:00,2,คำกล่าวของนางดาลี (Daly) สมาชิกคณะกรรมการนโยบา...
7,JPY,2025/05/21,06:00:00,1,ดัชนีจากการสำรวจ Reuters Tankan ( พ.ค.)
8,JPY,2025/05/21,06:50:00,2,ดุลการค้าที่ได้ปรับรายการ
9,JPY,2025/05/21,06:50:00,2,ปริมาณการส่งออก (ปีต่อปี) (เม.ย.)


In [7]:
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
news_df.to_csv(f'Data/Event_Calendar/{date_time}.csv', index=False)